<a href="https://colab.research.google.com/github/indl1670/KST_Project/blob/maskrcnn/mrcnn_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/matterport/Mask_RCNN.git '/content/drive/MyDrive/kst_project/oneCycle/Mask_RCNN2'

Cloning into '/content/drive/MyDrive/kst_project/oneCycle/Mask_RCNN2'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 125.23 MiB | 15.73 MiB/s, done.
Resolving deltas: 100% (565/565), done.
Checking out files: 100% (76/76), done.


In [ ]:
!pip3 install tensorboard==1.15.0 tensorflow==1.15.0 tensorflow-estimator==1.15.1 tensorflow-gpu==1.15.2 tensorflow-gpu-estimator==2.1.0 Keras==2.2.5 Keras-Applications==1.0.8 Keras-Preprocessing==1.1.0

     |████████████████████████████████| 3.8 MB 7.2 MB/s 
     |████████████████████████████████| 412.3 MB 24 kB/s 
     |████████████████████████████████| 503 kB 68.7 MB/s 
     |████████████████████████████████| 410.9 MB 35 kB/s 
     |████████████████████████████████| 464 kB 60.2 MB/s 
     |████████████████████████████████| 336 kB 56.8 MB/s 
     |████████████████████████████████| 50 kB 7.5 MB/s 
     |████████████████████████████████| 41 kB 873 kB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=52be0fb90cf0bcadc06c3c12a22de40e5ea455a5722ffbee7b28667e5fd10885
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing in

In [ ]:
!pip3 install h5py==2.10.0 --force-reinstall

     |████████████████████████████████| 2.9 MB 7.4 MB/s 
     |████████████████████████████████| 15.7 MB 51.5 MB/s 
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.5 which is incompatible.
tensorflow-probability 0.15.0 requires gast>=0.3.2, but you have gast 0.2.2 which is incompatible.
kapre 0.3.6 requires tensorflow>=2.0.0, but you have tensorflow 1.15.0 

In [ ]:
!python3 '/content/drive/MyDrive/kst_project/oneCycle/Mask_RCNN2/samples/coco/coco.py' train --dataset='/content/drive/Shareddrives/KST_Project/final_img/dataset' --model=coco

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/usr/local/lib/python3.7/dist-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/content/drive/MyDrive/kst_project/oneCycle/Mask_RCNN2/samples/coco/coco.py:265: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.stack(instance_masks, axis=2).astype(np.bool)
/usr/local/lib/python3.7/dist-packages/skimage/transform/_warps.py:830: Futu

In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory 경로 저장
ROOT_DIR = os.path.abspath("/content/drive/MyDrive/kst_project/oneCycle/Mask_RCNN/")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils

# MS-COCO 기반으로 Pretrained 된 모델을 로딩
import mrcnn.model as modellib
from mrcnn import visualize

# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# MRCNN 모델 경로 저장
MODEL_DIR = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# 자체 데이터셋 trained weight file 경로 저장
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "logs/mask_rcnn_kst_case2_0006.h5")

# 이미지 경로 저장
IMG_DIR = os.path.abspath("/content/drive/Shareddrives/KST_Project/final_img/")

# 테스트 이미지폴더 경로 저장
IMAGE_DIR = os.path.join(IMG_DIR, "dataset/test2021/")

class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

# config 세부사항 출력
config = InferenceConfig()
config.display()

# inference 모드에서 모델 객체 생성
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# MS-COCO에서 trained weights 로드
model.load_weights(
    COCO_MODEL_PATH, by_name=True)

# test2021(이미지 폴더)에서 무작위 이미지 선정
file_names = next(os.walk(IMAGE_DIR))[2]
image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))

# class명 저장(0: 'border_stone', 1: 'side_sphere', 2: 'Manhole', 3: 'center_separato', 4: 'PE_barrier', 5: 'temporary_safety_barrier')
class_names = ['border_stone', 'side_sphere', 'Manhole', 'center_separato', 'PE_barrier', 'temporary_safety_barrier']

# Detection
results = model.detect([image], verbose=1)

# 발견된 객체 개수 확인
print(results[0]['class_ids'].shape) 

# Visualize 결과
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])